## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!
import numpy as np
import pandas as pd
#from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

#### Overview of Data

In [32]:
#Load Dataset from Chicago Data Protal
cr_crash = pd.read_csv("Traffic_Crashes_-_Crashes.csv",low_memory=False)
cr_crash.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,Boundaries - ZIP Codes
0,006de7cd2f3d72f38ac1c0c062e3598a4f1991e726676a...,NaN,NaN,09/27/2023 10:15:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,0.0,2.0,0.0,22,4,9,41.890910,-87.663000,POINT (-87.663000492474 41.890910198581),49.0
1,c62af948baf96bbca7327bec50e8fe03638ae77c6d44af...,NaN,NaN,09/27/2023 09:30:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,0.0,2.0,0.0,21,4,9,41.718140,-87.651966,POINT (-87.651966170053 41.71813959268),13.0
2,55236fdde55d10b4d21bad23e148c8ae7ba4d4fa4b3270...,NaN,NaN,09/27/2023 09:25:00 PM,15,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,0.0,2.0,0.0,21,4,9,41.693109,-87.612230,POINT (-87.612230489331 41.693109211044),19.0
3,834faef54b92f0b8a797cc2857efc0ffc1fe1aaf8dde76...,NaN,Y,09/27/2023 09:25:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,FIXED OBJECT,...,0.0,1.0,0.0,21,4,9,41.939154,-87.822533,POINT (-87.82253268684 41.939154093727),52.0
4,d4b65fc56e0fbff63753cbc24a876c670d4e0f76be23d9...,NaN,NaN,09/27/2023 09:10:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,...,0.0,1.0,0.0,21,4,9,41.986820,-87.789000,POINT (-87.788999818498 41.986820039189),17.0


#### Data Information 

In [42]:
cr_crash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765268 entries, 0 to 765267
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                765268 non-null  object 
 1   RD_NO                          760783 non-null  object 
 2   CRASH_DATE_EST_I               57650 non-null   object 
 3   CRASH_DATE                     765268 non-null  object 
 4   POSTED_SPEED_LIMIT             765268 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         765268 non-null  object 
 6   DEVICE_CONDITION               765268 non-null  object 
 7   WEATHER_CONDITION              765268 non-null  object 
 8   LIGHTING_CONDITION             765268 non-null  object 
 9   FIRST_CRASH_TYPE               765268 non-null  object 
 10  TRAFFICWAY_TYPE                765268 non-null  object 
 11  LANE_CNT                       199004 non-null  float64
 12  ALIGNMENT                     

#### Value Counts of Selected Columns for Study

In [7]:
#Get value counts to find best columns to use as predictive variables
cr_crash["STREET_NAME"].value_counts().head(75)

WESTERN AVE         20875
PULASKI RD          18334
CICERO AVE          17134
ASHLAND AVE         16653
HALSTED ST          14866
                    ...  
83RD ST              2625
CENTRAL PARK AVE     2585
NARRAGANSETT AVE     2531
CANAL ST             2474
JEFFERY BLVD         2471
Name: STREET_NAME, Length: 75, dtype: int64

#### Logistic Regression Model

In [3]:
#Get value counts of the target variable - Need to balance this variable
cr_crash["CRASH_TYPE"].value_counts(normalize=True)

NO INJURY / DRIVE AWAY              0.734839
INJURY AND / OR TOW DUE TO CRASH    0.265161
Name: CRASH_TYPE, dtype: float64

In [33]:
# Create dataframe for columns of interest (predictive columns)
#"MOST_SEVERE_INJURY","RD_NO","REPORT_TYPE","LOCATION","Boundaries - ZIP Codes"
cr_crash = cr_crash[["ALIGNMENT","WEATHER_CONDITION","LIGHTING_CONDITION","TRAFFICWAY_TYPE","FIRST_CRASH_TYPE","DEVICE_CONDITION","POSTED_SPEED_LIMIT","DAMAGE","PRIM_CONTRIBUTORY_CAUSE"
                     ,"CRASH_MONTH","CRASH_DAY_OF_WEEK","CRASH_HOUR","ROADWAY_SURFACE_COND","STREET_NO","TRAFFIC_CONTROL_DEVICE","NUM_UNITS","SEC_CONTRIBUTORY_CAUSE",
                "INJURIES_UNKNOWN","INJURIES_NO_INDICATION","INJURIES_REPORTED_NOT_EVIDENT","INJURIES_NON_INCAPACITATING","INJURIES_INCAPACITATING",
                  "INJURIES_FATAL","INJURIES_TOTAL","NUM_UNITS","ROAD_DEFECT","CRASH_TYPE"]]
cr_crash

,ALIGNMENT,WEATHER_CONDITION,LIGHTING_CONDITION,TRAFFICWAY_TYPE,FIRST_CRASH_TYPE,DEVICE_CONDITION,POSTED_SPEED_LIMIT,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,CRASH_MONTH,...,INJURIES_UNKNOWN,INJURIES_NO_INDICATION,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NON_INCAPACITATING,INJURIES_INCAPACITATING,INJURIES_FATAL,INJURIES_TOTAL,NUM_UNITS,ROAD_DEFECT,CRASH_TYPE
0,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",DIVIDED - W/MEDIAN (NOT RAISED),ANGLE,NO CONTROLS,35,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,NO INJURY / DRIVE AWAY
1,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",T-INTERSECTION,ANGLE,FUNCTIONING PROPERLY,30,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,UNKNOWN,NO INJURY / DRIVE AWAY
2,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",UNKNOWN INTERSECTION TYPE,ANGLE,NO CONTROLS,15,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH
3,STRAIGHT AND LEVEL,CLEAR,DAYLIGHT,FOUR WAY,FIXED OBJECT,FUNCTIONING PROPERLY,30,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,UNKNOWN,NO INJURY / DRIVE AWAY
4,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",DIVIDED - W/MEDIAN (NOT RAISED),PARKED MOTOR VEHICLE,NO CONTROLS,30,"OVER $1,500",DISTRACTION - FROM INSIDE VEHICLE,9,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765263,STRAIGHT AND LEVEL,RAIN,DAYLIGHT,NOT DIVIDED,TURNING,FUNCTIONING PROPERLY,30,"$501 - $1,500",UNABLE TO DETERMINE,2,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,NO INJURY / DRIVE AWAY
765264,STRAIGHT AND LEVEL,CLEAR,DAYLIGHT,DIVIDED - W/MEDIAN (NOT RAISED),ANGLE,NO CONTROLS,30,"OVER $1,500",FAILING TO YIELD RIGHT-OF-WAY,1,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH
765265,STRAIGHT AND LEVEL,CLEAR,DUSK,DIVIDED - W/MEDIAN BARRIER,PARKED MOTOR VEHICLE,NO CONTROLS,30,"OVER $1,500",UNABLE TO DETERMINE,1,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,NO INJURY / DRIVE AWAY
765266,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",NOT DIVIDED,PEDESTRIAN,NO CONTROLS,30,"$501 - $1,500",UNABLE TO DETERMINE,6,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH


In [34]:
# Fill NA rows with number of highest frequency 
cr_crash["INJURIES_UNKNOWN"].fillna(cr_crash["INJURIES_UNKNOWN"]==0.0, inplace=True)
cr_crash["INJURIES_UNKNOWN"]
cr_crash["INJURIES_NO_INDICATION"].fillna(cr_crash["INJURIES_NO_INDICATION"]==2.0, inplace=True)
cr_crash["INJURIES_NO_INDICATION"]
cr_crash["INJURIES_REPORTED_NOT_EVIDENT"].fillna(cr_crash["INJURIES_REPORTED_NOT_EVIDENT"]==0.0, inplace=True)
cr_crash["INJURIES_REPORTED_NOT_EVIDENT"]
cr_crash["INJURIES_NON_INCAPACITATING"].fillna(cr_crash["INJURIES_NON_INCAPACITATING"]==0.0, inplace=True)
cr_crash["INJURIES_NON_INCAPACITATING"]
cr_crash["INJURIES_INCAPACITATING"].fillna(cr_crash["INJURIES_INCAPACITATING"]==0.0, inplace=True)
cr_crash["INJURIES_INCAPACITATING"]
cr_crash["INJURIES_FATAL"].fillna(cr_crash["INJURIES_FATAL"]==0.0, inplace=True)
cr_crash["INJURIES_FATAL"]
cr_crash["INJURIES_TOTAL"].fillna(cr_crash["INJURIES_TOTAL"]==0.0, inplace=True)
cr_crash["INJURIES_TOTAL"]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
765263    0.0
765264    1.0
765265    0.0
765266    1.0
765267    2.0
Name: INJURIES_TOTAL, Length: 765268, dtype: object

In [35]:
# Make sure Na's are filled
cr_crash.isna().sum()

ALIGNMENT                        0
WEATHER_CONDITION                0
LIGHTING_CONDITION               0
TRAFFICWAY_TYPE                  0
FIRST_CRASH_TYPE                 0
DEVICE_CONDITION                 0
POSTED_SPEED_LIMIT               0
DAMAGE                           0
PRIM_CONTRIBUTORY_CAUSE          0
CRASH_MONTH                      0
CRASH_DAY_OF_WEEK                0
CRASH_HOUR                       0
ROADWAY_SURFACE_COND             0
STREET_NO                        0
TRAFFIC_CONTROL_DEVICE           0
NUM_UNITS                        0
SEC_CONTRIBUTORY_CAUSE           0
INJURIES_UNKNOWN                 0
INJURIES_NO_INDICATION           0
INJURIES_REPORTED_NOT_EVIDENT    0
INJURIES_NON_INCAPACITATING      0
INJURIES_INCAPACITATING          0
INJURIES_FATAL                   0
INJURIES_TOTAL                   0
NUM_UNITS                        0
ROAD_DEFECT                      0
CRASH_TYPE                       0
dtype: int64

In [ ]:
#["RD_NO","CRASH_DATE","ROAD_DEFECT","REPORT_TYPE","MOST_SEVERE_INJURY","LOCATION","TRAFFIC_CONTROL_DEVICE"]

In [36]:
from sklearn.preprocessing import LabelEncoder

# Use Label Encoder to covert target variable to 0 and 1
label_encoder = LabelEncoder()
encoded_data = label_encoder.fit_transform(cr_crash["CRASH_TYPE"])
encoded_data
cr_crash["CRASH_TYPE"] = pd.DataFrame(encoded_data, columns= ["CRASH_TYPE"], index=cr_crash["CRASH_TYPE"].index)
cr_crash

,ALIGNMENT,WEATHER_CONDITION,LIGHTING_CONDITION,TRAFFICWAY_TYPE,FIRST_CRASH_TYPE,DEVICE_CONDITION,POSTED_SPEED_LIMIT,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,CRASH_MONTH,...,INJURIES_UNKNOWN,INJURIES_NO_INDICATION,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NON_INCAPACITATING,INJURIES_INCAPACITATING,INJURIES_FATAL,INJURIES_TOTAL,NUM_UNITS,ROAD_DEFECT,CRASH_TYPE
0,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",DIVIDED - W/MEDIAN (NOT RAISED),ANGLE,NO CONTROLS,35,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,1
1,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",T-INTERSECTION,ANGLE,FUNCTIONING PROPERLY,30,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,UNKNOWN,1
2,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",UNKNOWN INTERSECTION TYPE,ANGLE,NO CONTROLS,15,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,0
3,STRAIGHT AND LEVEL,CLEAR,DAYLIGHT,FOUR WAY,FIXED OBJECT,FUNCTIONING PROPERLY,30,"OVER $1,500",UNABLE TO DETERMINE,9,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,UNKNOWN,1
4,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",DIVIDED - W/MEDIAN (NOT RAISED),PARKED MOTOR VEHICLE,NO CONTROLS,30,"OVER $1,500",DISTRACTION - FROM INSIDE VEHICLE,9,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,NO DEFECTS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765263,STRAIGHT AND LEVEL,RAIN,DAYLIGHT,NOT DIVIDED,TURNING,FUNCTIONING PROPERLY,30,"$501 - $1,500",UNABLE TO DETERMINE,2,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,1
765264,STRAIGHT AND LEVEL,CLEAR,DAYLIGHT,DIVIDED - W/MEDIAN (NOT RAISED),ANGLE,NO CONTROLS,30,"OVER $1,500",FAILING TO YIELD RIGHT-OF-WAY,1,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2,NO DEFECTS,0
765265,STRAIGHT AND LEVEL,CLEAR,DUSK,DIVIDED - W/MEDIAN BARRIER,PARKED MOTOR VEHICLE,NO CONTROLS,30,"OVER $1,500",UNABLE TO DETERMINE,1,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,NO DEFECTS,1
765266,STRAIGHT AND LEVEL,CLEAR,"DARKNESS, LIGHTED ROAD",NOT DIVIDED,PEDESTRIAN,NO CONTROLS,30,"$501 - $1,500",UNABLE TO DETERMINE,6,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2,NO DEFECTS,0


#### Balance Target Variable

In [37]:
cr_crash.iloc[:,-1].value_counts()

1    562349
0    202919
Name: CRASH_TYPE, dtype: int64

In [38]:
ones = cr_crash[cr_crash.iloc[:,-1]== 1]
zeros = cr_crash[cr_crash.iloc[:,-1]== 0]

In [39]:
ones.iloc[:202919,-1].value_counts()

1    202919
Name: CRASH_TYPE, dtype: int64

In [40]:
onesdf = ones.iloc[:202919, :]

In [41]:
onesdf.shape

(202919, 27)

In [42]:
zeros.shape

(202919, 27)

In [43]:
cr_crash_ = pd.concat([onesdf,zeros],axis=0)
cr_crash_.shape

(405838, 27)

In [44]:
y = cr_crash_['CRASH_TYPE']
X = cr_crash_.drop(columns=['CRASH_TYPE'], axis=1)
X.head()
y.value_counts(normalize=True)

1    0.5
0    0.5
Name: CRASH_TYPE, dtype: float64

#### Train and Test Split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.columns

Index(['ALIGNMENT', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
       'TRAFFICWAY_TYPE', 'FIRST_CRASH_TYPE', 'DEVICE_CONDITION',
       'POSTED_SPEED_LIMIT', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE',
       'CRASH_MONTH', 'CRASH_DAY_OF_WEEK', 'CRASH_HOUR',
       'ROADWAY_SURFACE_COND', 'STREET_NO', 'TRAFFIC_CONTROL_DEVICE',
       'NUM_UNITS', 'SEC_CONTRIBUTORY_CAUSE', 'INJURIES_UNKNOWN',
       'INJURIES_NO_INDICATION', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_INCAPACITATING',
       'INJURIES_FATAL', 'INJURIES_TOTAL', 'NUM_UNITS', 'ROAD_DEFECT'],
      dtype='object')

In [46]:
y_train.value_counts(normalize=True)

0    0.500808
1    0.499192
Name: CRASH_TYPE, dtype: float64

#### One Hot Encode Object Variables

In [47]:
numbers = cr_crash[["POSTED_SPEED_LIMIT","CRASH_MONTH","CRASH_DAY_OF_WEEK","NUM_UNITS","CRASH_HOUR","STREET_NO","INJURIES_UNKNOWN","INJURIES_NO_INDICATION","INJURIES_REPORTED_NOT_EVIDENT","INJURIES_NON_INCAPACITATING","INJURIES_INCAPACITATING",
                  "INJURIES_FATAL","INJURIES_TOTAL"]]
numbers
# preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# model validation: testing model variance with cross validation

from sklearn.model_selection import cross_val_score, cross_validate
#numbers = cr_crash[["POSTED_SPEED_LIMIT","CRASH_MONTH","CRASH_DAY_OF_WEEK","CRASH_HOUR","STREET_NO","BEAT_OF_OCCURRENCE"]]
#numbers
# make only cat data frame and one hot encoding it. it will give you an array make the onehot array into dataframe

ohe = OneHotEncoder(drop='first')
dummies = ohe.fit_transform(X_train[["ALIGNMENT","WEATHER_CONDITION","LIGHTING_CONDITION","TRAFFIC_CONTROL_DEVICE","ROAD_DEFECT","SEC_CONTRIBUTORY_CAUSE","TRAFFICWAY_TYPE","FIRST_CRASH_TYPE","DEVICE_CONDITION","DAMAGE","PRIM_CONTRIBUTORY_CAUSE",
                   "ROADWAY_SURFACE_COND"]])

# Getting a DF
X_train_onehot = pd.DataFrame(dummies.todense(), columns=ohe.get_feature_names_out(), index=X_train.index)


# concat number and one dataframe
X_train_ = pd.concat([numbers,X_train_onehot],axis=1)
X_train_.dropna(inplace=True)
#X_train_ = X_train_.dropna(subset = [])
X_train_

,POSTED_SPEED_LIMIT,CRASH_MONTH,CRASH_DAY_OF_WEEK,NUM_UNITS,NUM_UNITS,CRASH_HOUR,STREET_NO,INJURIES_UNKNOWN,INJURIES_NO_INDICATION,INJURIES_REPORTED_NOT_EVIDENT,...,PRIM_CONTRIBUTORY_CAUSE_UNABLE TO DETERMINE,PRIM_CONTRIBUTORY_CAUSE_UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED),"PRIM_CONTRIBUTORY_CAUSE_VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)",PRIM_CONTRIBUTORY_CAUSE_WEATHER,ROADWAY_SURFACE_COND_ICE,ROADWAY_SURFACE_COND_OTHER,"ROADWAY_SURFACE_COND_SAND, MUD, DIRT",ROADWAY_SURFACE_COND_SNOW OR SLUSH,ROADWAY_SURFACE_COND_UNKNOWN,ROADWAY_SURFACE_COND_WET
0,35,9,4,2,2,22,1419,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30,9,4,2,2,21,9700,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,9,4,2,2,21,427,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30,9,4,1,1,21,3249,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30,9,4,5,5,21,5777,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765228,30,8,3,2,2,8,4300,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
765251,30,5,2,2,2,23,215,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
765264,30,1,3,2,2,7,7400,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
765266,30,6,7,2,2,20,2358,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Transform Test Object Variables

In [48]:
test_dummies = ohe.transform(X_test[["ALIGNMENT","WEATHER_CONDITION","LIGHTING_CONDITION","TRAFFIC_CONTROL_DEVICE","ROAD_DEFECT","SEC_CONTRIBUTORY_CAUSE","TRAFFICWAY_TYPE","FIRST_CRASH_TYPE","DEVICE_CONDITION","DAMAGE","PRIM_CONTRIBUTORY_CAUSE",
                   "ROADWAY_SURFACE_COND"]])
test_ = pd.DataFrame(test_dummies.todense(), columns=ohe.get_feature_names_out(),
                       index=X_test.index)
X_test_ = pd.concat([X_test[["POSTED_SPEED_LIMIT","CRASH_MONTH","CRASH_DAY_OF_WEEK","NUM_UNITS","CRASH_HOUR","STREET_NO","INJURIES_UNKNOWN","INJURIES_NO_INDICATION","INJURIES_REPORTED_NOT_EVIDENT","INJURIES_NON_INCAPACITATING","INJURIES_INCAPACITATING",
                  "INJURIES_FATAL","INJURIES_TOTAL"]], test_], axis=1)
X_test_

,POSTED_SPEED_LIMIT,CRASH_MONTH,CRASH_DAY_OF_WEEK,NUM_UNITS,NUM_UNITS,CRASH_HOUR,STREET_NO,INJURIES_UNKNOWN,INJURIES_NO_INDICATION,INJURIES_REPORTED_NOT_EVIDENT,...,PRIM_CONTRIBUTORY_CAUSE_UNABLE TO DETERMINE,PRIM_CONTRIBUTORY_CAUSE_UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED),"PRIM_CONTRIBUTORY_CAUSE_VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)",PRIM_CONTRIBUTORY_CAUSE_WEATHER,ROADWAY_SURFACE_COND_ICE,ROADWAY_SURFACE_COND_OTHER,"ROADWAY_SURFACE_COND_SAND, MUD, DIRT",ROADWAY_SURFACE_COND_SNOW OR SLUSH,ROADWAY_SURFACE_COND_UNKNOWN,ROADWAY_SURFACE_COND_WET
158416,30,4,1,2,2,22,1600,0.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114648,30,9,3,3,3,16,3046,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269811,30,4,4,2,2,23,2400,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37265,30,5,4,2,2,16,4120,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159139,25,4,6,2,2,17,1051,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273031,35,4,6,2,2,12,832,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
464170,30,5,4,2,2,8,230,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172076,30,3,2,2,2,4,8631,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
551622,30,8,3,2,2,17,499,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Baseline Logistic Regression Model

In [49]:
# Run the Logistic Regression Model and fit the model to the Train data
logreg = LogisticRegression() #(fit_intercept=False, C=1e12, solver='liblinear')

logreg.fit(X_train_, y_train)

LogisticRegression()

In [50]:
logreg.score(X_train_, y_train)

0.5030061305350584

In [51]:
y_hat_train = logreg.predict(X_train_)
y_hat_test = logreg.predict(X_test_)

#### Train Data Results

In [52]:
residuals = np.abs(y_train - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True)) 

0    153104
1    151274
Name: CRASH_TYPE, dtype: int64
0    0.503006
1    0.496994
Name: CRASH_TYPE, dtype: float64


In [ ]:
# This is a 50% accuracy 

#### Test Data Results

In [53]:
residuals = np.abs(y_test - y_hat_test)
print(pd.Series(residuals).value_counts())
print('------------------------------------')
print(pd.Series(residuals).value_counts(normalize=True))

1    52120
0    49340
Name: CRASH_TYPE, dtype: int64
------------------------------------
1    0.5137
0    0.4863
Name: CRASH_TYPE, dtype: float64


#### Initial Baseline Results

The initial baseline results show that the train data haas a 50.3% accuracy and the test data has a 51.37% accuracy. This number is relatively low compared to what is considered to be a perfect acccuracy score of 1.0. The takeaway from this initial result is that there is more that can be done with the data. One of the problems with that data is that there are too many unique values for the zipcode to be encoded and used. It also has a significant number of NAs. The inital recommendation here would be to go through the data even more and see how high you could get that accuracy level to be. Steps were made to add in more predictive variables filtering the NA values, however, it did not end up changing the result.

#### Hyperparameter Models

In [54]:
import pandas as pd
import numpy as np
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [55]:
C_param_range = [0.1, 1, 10, 100, 1000, 10000]
names = [0.1, 1, 10, 100, 1000, 10000]
colors = sns.color_palette('Set2')

plt.figure(figsize=(10, 8))

for n, c in enumerate(C_param_range):
    # Fit a model
    logreg = LogisticRegression(fit_intercept=False, C=c, solver='liblinear')
    model_log = logreg.fit(X_train_, y_train)
    print(model_log) # Preview model params

    # Predict
    y_hat_test = logreg.predict(X_test_)

    y_score = logreg.fit(X_train_, y_train).decision_function(X_test_)

    fpr, tpr, thresholds = roc_curve(y_test, y_score)
    
    print('AUC for {}: {}'.format(names[n], auc(fpr, tpr)))

LogisticRegression(C=0.1, fit_intercept=False, solver='liblinear')
AUC for 0.1: 0.46408750427065004
LogisticRegression(C=1, fit_intercept=False, solver='liblinear')
AUC for 1: 0.4646881110654265
LogisticRegression(C=10, fit_intercept=False, solver='liblinear')
AUC for 10: 0.454669504236654
LogisticRegression(C=100, fit_intercept=False, solver='liblinear')
AUC for 100: 0.45841638571086374
LogisticRegression(C=1000, fit_intercept=False, solver='liblinear')
AUC for 1000: 0.45493239534215263
LogisticRegression(C=10000, fit_intercept=False, solver='liblinear')
AUC for 10000: 0.46012411260442726


<Figure size 1000x800 with 0 Axes>

#### Cross Validation

In [56]:
cv_results = cross_validate(
                    estimator=logreg,
                    X=X_train_,
                    y=y_train,
                    cv=5,
                    return_train_score=True)
cv_results

{'fit_time': array([21.41608477, 11.16908908, 11.91400099, 12.02322578,  9.96285009]),
 'score_time': array([0.8421061 , 0.80462193, 0.83011293, 0.83852029, 0.81570888]),
 'test_score': array([0.50090348, 0.50019712, 0.4970596 , 0.50132238, 0.50304723]),
 'train_score': array([0.50984797, 0.50934284, 0.51113749, 0.51001836, 0.50908613])}

In [57]:
cv_results_2 = cross_validate(
                    estimator=logreg,
                    X=X_train_,
                    y=y_train,
                    cv=10,
                    return_train_score=True)
cv_results_2

{'fit_time': array([13.99661899, 16.98846483, 14.93198013, 13.72859693, 13.25433922,
        14.36438584, 17.8771441 , 13.49797606, 13.82443285, 13.18416214]),
 'score_time': array([0.44727802, 0.41248107, 0.40487599, 0.39133692, 0.39080381,
        0.39898801, 0.39724493, 0.40880084, 0.41347194, 0.41222501]),
 'test_score': array([0.50302254, 0.49967146, 0.50341678, 0.49799593, 0.49806163,
        0.49319929, 0.49710888, 0.50394244, 0.50093636, 0.50264481]),
 'train_score': array([0.5088596 , 0.50889611, 0.50868438, 0.50890706, 0.50808936,
        0.51018836, 0.50911513, 0.50886325, 0.50937246, 0.50881759])}

In [29]:
cv_results['train_score']

array([0.50080903, 0.50032443, 0.50010267, 0.50081929, 0.50363651])

In [30]:
cv_results['test_score']

array([0.50080491, 0.49921151, 0.49870228, 0.50022177, 0.50092813])

In [43]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_hat_train))
print('Testing Precision: ', precision_score(y_test, y_hat_test))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_hat_train))
print('Testing Recall: ', recall_score(y_test, y_hat_test))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_hat_train))
print('Testing Accuracy: ', accuracy_score(y_test, y_hat_test))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_hat_train))
print('Testing F1-Score: ', f1_score(y_test, y_hat_test))

Training Precision:  0.5028700943558917
Testing Precision:  0.48884904455469863



Training Recall:  0.403019553385151
Testing Recall:  0.3904386377903327



Training Accuracy:  0.5031046921919455
Testing Accuracy:  0.4886260595308496



Training F1-Score:  0.44744187405924385
Testing F1-Score:  0.4341367651870433


#### Decision Trees

In [44]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree

In [46]:
classifier = DecisionTreeClassifier(random_state=10)  
classifier.fit(X_train_, y_train)

DecisionTreeClassifier(random_state=10)

In [49]:
y_pred = classifier.predict(X_test_)  

In [50]:
# Calculate accuracy 
acc = accuracy_score(y_test,y_pred) * 100
print('Accuracy is :{0}'.format(acc))

# Check the AUC for predictions
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('\nAUC is :{0}'.format(round(roc_auc, 2)))

# Create and print a confusion matrix 
print('\nConfusion Matrix')
print('----------------')
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy is :50.00985610092648

AUC is :0.5

Confusion Matrix
----------------


Predicted,0,1,All
True,,,
0,25290,25194,50484
1,25526,25450,50976
All,50816,50644,101460
